# 大模型推理配置自动推荐

千帆平台上提供了大量模型，同时每个模型都提供了丰富的参数可供用户调整以适应不同的应用场景。然而，挑选合适的模型以及为模型挑选合适的参数是一件十分繁琐的事，需要不断进行尝试，并且在不同场景下，最优的配置也不尽相同，需要反复。

针对这个问题，SDK 提供了推理配置自动推荐的功能，只需提供您的目标场景数据集和评估标准，设定您的搜索空间，我们的SDK便能智能推荐最优的模型及其参数配置。这一全新功能确保了在不同场景下，您都能轻松找到或优化至理想的模型配置，释放模型潜能，提升效能至极致。

接下来，在本 cookbook 中将展示如何使用 SDK 中的推理配置推荐功能，并展示该功能所带来的性能提升。

**注意**：这个功能需要千帆 SDK 版本 >= 0.3.7

In [1]:
import os
import qianfan

os.environ["QIANFAN_ACCESS_KEY"] = "your access key"
os.environ["QIANFAN_SECRET_KEY"] = "your secret key"
# 由于后续在调优配置的过程中会并发请求模型，建议限制 QPS 和重试次数，避免调用失败
os.environ["QIANFAN_QPS_LIMIT"] = "3"
os.environ["QIANFAN_LLM_API_RETRY_COUNT"] = "5"

## 准备工作

具体而言，在获取推荐配置前，需要先准备：

- 数据集：根据目标场景准备一定量的数据
- 评估方式：根据目标场景，选择待优化的指标，并提供评估函数

数据集使用的是千帆 SDK 中提供的 `Dataset` 模块，可以直接加载本地的数据集文件，也可以使用平台上预置的或者自行上传的数据集，具体加载方式参考 [文档](https://github.com/baidubce/bce-qianfan-sdk/blob/main/docs/dataset.md)。

这里我们以一个 **角色扮演** 数据集为例

In [ ]:
from qianfan.dataset import Dataset

dataset = Dataset.load(
    data_file="./example.jsonl",
    organize_data_as_group=False,
    input_columns=["prompt"],
    reference_column="response",
)

In [6]:
for item in dataset[0]:
    print(item['prompt'])
    print(item['response'][0][0])

[INFO] [04-01 11:29:12] dataset.py:883 [t:140289325633792]: list local dataset data by 0


{'姓名': '白展堂', '性别': '男', '物种': '人类', '年龄': '25岁', '工作': '同福客栈跑堂、七侠镇捕快兼口头行动负责人（临时）、同福武馆馆长（已解散）', '昵称': '白玉汤、盗圣、展堂、老白、白玉展堂、堂堂（小名）、白大哥、白二哥（姬无病称呼）、黄豆豆、草头王、王豆豆、草头黄先生、干大爷、大舅哥、能喝八两绝不喝半斤、杀千刀的老贼、贼头、红鼻子盗圣、小白、展大哥、老白同志、白少侠、盗圣哥哥、我滴儿、白捕头、姐夫、大佬、白展胖、白先生、好女婿、乖女婿', '身高': '1.83 m', '体重': '140 斤', '居住地': '东北', '恋爱状态': '已婚，妻子是佟湘玉', '爱好': '钱（或者珠宝）、说书或听书、赌博（曾经）、唱歌、喝酒、画画、医术', '学历': '上等文化', '其他': '喜欢财宝，有贼性，但愿意为朋友两肋插刀，有一定的医术知识', '经典台词': ['天下第二，第一是楚留香，比赛那天我光着脚，而且是顶风。', '你说的这是我那没脸没皮好吃懒做的白大哥吗？', '紧张的时候咬手指，一个不够得啃俩。', '练武是用来比的吗？是在真正需要它的时候派上用场。强盗来了你可以迎头痛击，路见不平你可以拔刀相助。只有在这些时候，武功才能发挥出它真正的威力。而这威力的源泉，都是来源于你心中的一股正气。一个人，只要他有了这股正气，即使他没有武功，同样可以天下无敌！', '此恨绵绵无期绝~', '天若有情天亦老~', '我真是少爷的身子跑堂的命。', '站在天堂看地狱，人生就像情景剧，站在地狱看天堂，为谁辛苦为谁忙。', '分手总是在雨天!', '客官，首先请接受我诚挚的祝福', '只要给够加班费，当牛做马无所谓!', '找点哪!', '葵花点穴手!', '再来劲小心我点你'], '口头禅': ['客官您里边请!', '此恨绵绵无期绝~', '天若有情天亦老~', '找点哪!', '葵花点穴手!'], '人物经历': '白展堂，原名白玉汤，葵花派弟子，江湖上的盗圣。后厌倦江湖生活，落脚同福客栈，改名白展堂。曾有一段初恋情感经历，最终与佟湘玉结为夫妻。在同福客栈中，他以其独特的个性和机智幽默成为了不可或缺的一员。', '人物关系': '父亲周氏，母亲白三娘（白翠萍），妻子佟湘玉，师父葵花派东西南北四大长老，师妹祝无双，

评估采用的 SDK 提供的 Evaluator 模块，基于 Evaluator 实现 evaluate 方法即可。如下实现了一个利用大模型评分实现评估的 Evaluator，关于如何实现 Evaluator 可以参考 [该cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/evaluation/local_eval_with_qianfan.ipynb)。

In [9]:
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan import ChatCompletion
from qianfan.common.prompt.prompt import Prompt
from qianfan.utils.pydantic import Field

from typing import Optional, Union, Any, Dict, List
import re
import json

class LocalJudgeEvaluator(LocalEvaluator):
    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    cache: Dict[str, Any] = Field(default={}, description="cache for evaluation")
    eval_prompt: Prompt = Field(
        default=Prompt(
            template="""你需要扮演一个裁判的角色，对一段角色扮演的对话内容进行打分，你需要考虑这段文本中的角色沉浸度和对话文本的通畅程度。你可以根据以下规则来进行打分，你可以阐述你对打分标准的理解后再给出分数：
                "4":完全可以扮演提问中的角色进行对话，回答完全符合角色口吻和身份，文本流畅语句通顺
                "3":扮演了提问中正确的角色，回答完全符合角色口吻和身份，但文本不流畅或字数不满足要求
                "2":扮演了提问中正确的角色，但是部分语句不符合角色口吻和身份，文本流畅语句通顺
                "1":能够以角色的口吻和身份进行一部分对话，和角色设定有一定偏差，回答内容不流畅，或不满足文本字数要求
                "0":扮演了错误的角色，没有扮演正确的角色，角色设定和提问设定差异极大，完全不满意
                你的回答需要以json代码格式输出：
                ```json
                {"modelA": {"justification": "此处阐述对打分标准的理解", "score": "此处填写打分结果"}}
                ```

                现在你可以开始回答了：
                问题：{{input}}
                ---
                modelA回答：{{output}}
                ---""",
            identifier="{{}}",
        ),
        description="evaluation prompt",
    )

    class Config:
        arbitrary_types_allowed = True

    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        score = 0
        try:
            # 渲染评估用的 prompt，传入输入、模型输出和参考答案
            p, _ = self.eval_prompt.render(
                **{
                    "input": "\n".join([i["content"] for i in input[1:]]),
                    "output": output,
                    "expect": reference,
                }
            )
            # 利用 cache 避免对同一结果反复进行评估，提升效率
            if p in self.cache:
                model_output = self.cache[p]
                score = float(model_output["modelA"]["score"])
            else:
                # 请求模型进行评估
                r = self.model.do(messages=[{"role": "user", "content": p}], temperature=0.01)
                content = r["result"]
                model_output = content
                # 提取出 json 格式的评估结果
                regex = re.compile("\`\`\`json(.*)\`\`\`", re.MULTILINE | re.DOTALL)
    
                u = regex.findall(content)
    
                if len(u) == 0:
                    score = 0
                else:
                    model_output = json.loads(u[0])
                    score = float(model_output["modelA"]["score"])
                    self.cache[p] = model_output
        except Exception as e:
            score = 0
        # 返回评估结果，这里字段需与后续推荐配置时设定的评估字段一致
        return {"score": score}

## 获取推荐配置

为了获取推荐配置，还需要设置一个超参搜索空间，千帆 SDK 提供了如下表示搜索空间的类：

- `Uniform`：表示一个均匀分布的搜索空间，包含两个参数 `low` 和 `high`，分别表示下界和上界。
- `Categorical`：表示一个离散的搜索空间，包含一个参数 `choices`，表示一组候选值。

In [10]:
from qianfan.autotuner.space import Uniform, Categorical

search_space = {
    "temperature": Uniform(0.01, 0.99),  # 设定temperature的范围
    "model": Categorical(["ERNIE-Speed", "ERNIE-Bot-turbo"]),  # 设定model的取值范围
    # 更多其他参数也可以按同样方式设定
}

之后就可以执行推荐

In [ ]:
import qianfan.autotuner

context = await qianfan.autotuner.run(
    search_space=search_space,
    dataset=dataset,
    evaluator=LocalJudgeEvaluator(
        model=ChatCompletion(model="ERNIE-Bot-4")
    ),
    # 以下均为可选参数
    suggestor="random",  # 搜索算法，目前仅支持 "random"，更多算法敬请期待
    cost_budget=20,      # 设定整个流程的预算，达到预算则终止流程，单位为 “元”
    metrics="score",     # 设定评估指标字段，与 Evaluator 输出对应
    mode="max",          # 设定评估指标最大化还是最小化
    repeat=5,            # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    # max_turn=10,         # 设定最大尝试次数
    # max_time=3600,       # 设定最大尝试时间，单位为秒
    log_dir= "./log",    # 日志目录
)

返回的结果是一个 `Context` 对象，其中包含了整个搜索过程的所有上下文信息，例如可以通过如下方式获得搜索的最佳参数

In [28]:
context.best

{'temperature': 0.23879676911568554, 'model': 'ERNIE-Speed'}

这个参数可以直接用于推理

In [ ]:
chat = qianfan.ChatCompletion().do(messages=[{
    "role": "user",
    "messages": "请扮演一个角色，然后说一句话"
}], **context.best)

print(chat['result'])

context 中也包含了整个过程中尝试的记录，可以获取某一轮某一组配置的评估结果等信息

In [29]:
for turn in context.history:
    for trial in turn:
        metrics = trial.metrics
        config = trial.config
        print("{}\t{}\t{}".format(config['model'], config['temperature'], metrics['score']))

ERNIE-Bot-turbo	0.9350118283831285	2.1816326530612247
ERNIE-Bot-turbo	0.37900692272161335	2.6857142857142855
ERNIE-Speed	0.5047393945107158	2.8081632653061224
ERNIE-Speed	0.23879676911568554	3.020408163265306
ERNIE-Speed	0.8166117015206144	2.5408163265306123
ERNIE-Bot-turbo	0.9271271943982626	2.1918367346938776
ERNIE-Bot-turbo	0.8600257278657117	2.23265306122449
ERNIE-Bot-turbo	0.3153142670496584	2.563265306122449
ERNIE-Speed	0.47733409897219253	2.8183673469387753
ERNIE-Bot-turbo	0.9237762484049076	2.263265306122449
ERNIE-Bot-turbo	0.9173899230454107	2.4244897959183676
ERNIE-Bot-turbo	0.7079405253554338	2.4959183673469387
ERNIE-Bot-turbo	0.4197475117892281	2.673469387755102
ERNIE-Bot-turbo	0.8722063351310373	2.4244897959183676
ERNIE-Speed	0.3138439582719443	2.8959183673469386
ERNIE-Speed	0.448023350389325	2.8224489795918366
ERNIE-Speed	0.9165385520594768	2.559183673469388
ERNIE-Bot-turbo	0.8672876273361033	2.4653061224489794
ERNIE-Speed	0.4456239330358356	2.840816326530612
ERNIE-Bot-tu

# 效果评估

从上面可以看到，搜索出的最佳配置 `{'temperature': 0.23879676911568554, 'model': 'ERNIE-Speed'}` 对应的分数约为 3.02。

为了评估 SDK 推荐参数的性能表现，接下来我们将参考默认参数的 `ERNIE-Speed` 表现作为 baseline，而这数据也可以通过 SDK 快速获得，只需要调整搜索空间即可

In [18]:
baseline = await qianfan.autotuner.run(
    search_space={
        "model": Categorical(["ERNIE-Speed"]),  # 这里仅保留唯一参数
    },
    dataset=dataset,
    evaluator=LocalJudgeEvaluator(
        model=ChatCompletion(model="ERNIE-Bot-4")
    ),
    # 以下参数与上述保持一致
    suggestor="random",  # 搜索算法，目前仅支持 "random"，更多算法敬请期待
    metrics="score",     # 设定评估指标字段，与 Evaluator 输出对应
    mode="max",          # 设定评估指标最大化还是最小化
    repeat=5,            # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    max_turn=1,          # 设定最大尝试次数
    log_dir= "./log",    # 日志目录
)

[INFO] [04-01 14:08:19] launcher.py:108 [t:140289325633792]: turn 0 started...
[INFO] [04-01 14:08:19] launcher.py:109 [t:140289325633792]: suggested config list: [{'model': 'ERNIE-Speed'}]
[INFO] [04-01 14:08:19] dataset.py:883 [t:140289325633792]: list local dataset data by None
[INFO] [04-01 14:08:19] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [04-01 14:08:19] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [04-01 14:08:19] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [04-01 14:08:19] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [04-01 14:08:19] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [04-01 14:08:19] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint

In [30]:
baseline.history[0][0].metrics['score']

2.4122448979591837

可以看到，在未进行参数调优时，模型得分仅为2.41，而调整参数后得分提升到了3.02，模型表现得到了巨大提升。

为进一步了解模型的性能，接下来我们尝试与默认参数的 ERNIE-3.5 进行对比。

In [20]:
eb35_result = await qianfan.autotuner.run(
    search_space={
        "model": Categorical(["ERNIE-3.5-8K"]),  
    },
    dataset=dataset,
    evaluator=LocalJudgeEvaluator(
        model=ChatCompletion(model="ERNIE-Bot-4")
    ),
    # 以下参数与上述保持一致
    suggestor="random",  # 搜索算法，目前仅支持 "random"，更多算法敬请期待
    metrics="score",     # 设定评估指标字段，与 Evaluator 输出对应
    mode="max",          # 设定评估指标最大化还是最小化
    repeat=5,            # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    max_turn=1,          # 设定最大尝试次数
    log_dir= "./log",    # 日志目录
)

[INFO] [04-01 14:21:57] launcher.py:108 [t:140289325633792]: turn 0 started...
[INFO] [04-01 14:21:57] launcher.py:109 [t:140289325633792]: suggested config list: [{'model': 'ERNIE-3.5-8K'}]
[INFO] [04-01 14:21:57] dataset.py:883 [t:140289325633792]: list local dataset data by None
[INFO] [04-01 14:21:57] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/completions
[INFO] [04-01 14:21:57] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/completions
[INFO] [04-01 14:21:57] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/completions
[INFO] [04-01 14:21:57] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/completions
[INFO] [04-01 14:21:57] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoint: /chat/completions
[INFO] [04-01 14:21:57] openapi_requestor.py:377 [t:140289325633792]: async requesting llm api endpoin

In [32]:
eb35_result.history[0][0].metrics['score']

3.0475206611570247

In [38]:
print("Avg Tokens Per Second")
print("ERNIE Speed: ", baseline.history[0][0].metrics['avg_tokens_per_second'])
print("ERNIE 3.5: ", eb35_result.history[0][0].metrics['avg_tokens_per_second'])

Avg Tokens Per Second
ERNIE Speed:  63.843757264518786
ERNIE 3.5:  54.79204768797332


可以看到推荐的参数配置 ERNIE Speed 表现已经能够与默认的 ERNIE 3.5 所媲美，而与此同时 ERNIE Speed 的价格仅为 ERNIE 3.5 的三分之一，能够在大幅降低成本的前提下保证性能几乎与更大的模型持平，还能获得更高的 token 吞吐量，有助于在实际应用场景下实现降本增效。